# Washington DOT Traffic API

In [3]:
import pandas as pd

import requests

## Import env variables

In [1]:
from dotenv import load_dotenv
import os

# load the .env file
load_dotenv()

# access the env variables
api_key = os.getenv("API_KEY")
db_url = os.getenv("DATABASE_URL")

# print to verify
# print(f"API Key: {api_key}")
# print(f"Database URL: {db_url}")

## Get API data

In [4]:
full_url = f"{db_url}{api_key}"

response = requests.get(full_url)

# check if request was successful
if response.status_code == 200:
    print("Data fetched successfully.")
    data = response.json()
    df = pd.DataFrame(data)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

Data fetched successfully.


In [6]:
df.head(50)

,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID
0,7,7,Marvin Rd./SR510 in Lacey To 14th Ave. in Olympia,6.62,"{'Description': 'I-5 @ 14th Ave in Olympia', '...","SB I-5, Lacey (SR510) to Olympia",{'Description': 'I-5 @ Marvin Rd I/C in Lacey'...,/Date(1737916800000-0800)/,116
1,6,6,JBLM Main Gate to Lakewood at SR512,6.25,"{'Description': 'I-5 @ SR512 in Lakewood', 'Di...","NB I-5, JBLM Main Gate to Lakewood (SR512)","{'Description': 'I-5 @ JBLM Main Gate NB', 'Di...",/Date(1737916800000-0800)/,120
2,6,6,SR512 in Lakewood To JBLM Main Gate,6.25,"{'Description': 'I-5 @ JLBM Main Gate', 'Direc...","SB I-5, Lakewood (SR512) to JBLM Main Gate","{'Description': 'I-5 @ SR512 in Lakewood SB', ...",/Date(1737916800000-0800)/,121
3,7,7,SR512 in Lakewood To Tacoma Dome in Tacoma,6.62,"{'Description': 'I-5 @ Tacoma Dome in Tacoma',...","NB I-5,Lakewood at SR512 To Tacoma","{'Description': 'I-5 @ SR512 in Lakewood NB', ...",/Date(1737916800000-0800)/,122
4,6,7,Tacoma Dome in Tacoma To SR512 in Lakewood,7.18,"{'Description': 'I-5 @ SR512 in Lakewood', 'Di...","SB I-5,Tacoma To Lakewood at SR512","{'Description': 'I-5 @ M St. in Tacoma SB', 'D...",/Date(1737916800000-0800)/,123
5,5,5,M St. in Tacoma To NB I-5 @ Pierce King County...,5.44,{'Description': 'I-5 @ Pierce King County Line...,"NB I-5,Tacoma To Pierce King County Line","{'Description': 'I-5 @ Tacoma Dome in Tacoma',...",/Date(1737916800000-0800)/,125
6,29,29,14th Ave in Olympia to Tacoma Dome in Tacoma,28.59,"{'Description': 'I-5 @ Tacoma Dome in Tacoma',...","NB I-5, Olympia To Tacoma","{'Description': 'I-5 @ 14th Ave in Olympia', '...",/Date(1737916800000-0800)/,126
7,10,0,Tacoma Dome in Tacoma To NB I-5 @ So. 320th St...,0.00,{'Description': 'I-5 @ So. 320th St in Federal...,"NB I-5,Tacoma To Federal Way","{'Description': 'I-5 @ Tacoma Dome in Tacoma',...",/Date(1737916800000-0800)/,303
8,7,7,Olympia to Marvin Rd SR510 in Lacey,6.62,{'Description': 'I-5 @ Marvin Rd I/C in Lacey'...,NB I-5 Olympia to Lacey (SR510),"{'Description': 'I-5 @ 14th Ave in Olympia', '...",/Date(1737916800000-0800)/,310
9,9,9,JBLM Main Gate to Marvin Rd SR510 in Lacey,9.10,{'Description': 'I-5 @ Marvin Rd SR510 in Lace...,SB I-5 JBLM Main Gate to Lacey (SR510) SB,"{'Description': 'I-5 @ JBLM Main Gate', 'Direc...",/Date(1737916800000-0800)/,311


In [9]:
df.shape

(168, 9)